Imports necessários

In [21]:
import requests
from io import BytesIO
import zipfile
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

Download e extração

In [ ]:
# URL do arquivo compactado
url_do_arquivo_zip = 'https://portaldatransparencia.gov.br/download-de-dados/viagens/2023'

# Baixando o arquivo compactado
resposta = requests.get(url_do_arquivo_zip)

# Verificando se a solicitação foi bem-sucedida
if resposta.status_code == 200:
    # Lendo o conteúdo do arquivo compactado
    conteudo_zip = BytesIO(resposta.content)

    # Extraindo o conteúdo do arquivo compactado
    with zipfile.ZipFile(conteudo_zip, 'r') as zip_ref:
        # Local da extracao
        zip_ref.extractall('/csv_files')
    print("Conteúdo extraído com sucesso!")
else:
    print(f"Falha ao baixar o arquivo. Código de status: {resposta.status_code}")

Leitura e transformacao dos arquivos .csv em DataFrame

In [58]:
arquivo_viagem = 'csv_files/2023_Viagem.csv'
arquivo_passagem = 'csv_files/2023_Passagem.csv'
arquivo_trecho = 'csv_files/2023_Trecho.csv'
arquivo_pagamento = 'csv_files/2023_Pagamento.csv'

# Carregando os dados do arquivo CSV em um DataFrame do pandas
df_viagem = pd.read_csv(arquivo_viagem, sep=';', encoding="latin-1")

#df_passagem = pd.read_csv(arquivo_passagem, sep=';', encoding="latin-1")

#df_trecho = pd.read_csv(arquivo_trecho, sep=';', encoding="latin-1")

#df_pagamento = pd.read_csv(arquivo_pagamento, sep=';', encoding="latin-1")


Tratamento dos dados do dataframe viagem

In [59]:
# Removendo colunas que não serão utilizadas
colunas_to_remove = ['Código do órgão superior', 'Nome do órgão superior', 'Código órgão solicitante', 'Nome órgão solicitante', 'Função', 'Descrição Função', 'Período - Data de início', 'Período - Data de fim', 'Destinos', 'Valor diárias', 'Valor passagens']
df_viagem.drop(colunas_to_remove, axis=1, inplace=True)

# Tratamento para as colunas com valores monetarios
colunas_to_numeric = ['Valor devolução', 'Valor outros gastos']
for coluna in colunas_to_numeric:
    df_viagem[coluna] = df_viagem[coluna].str.replace(',', '.')
    df_viagem[coluna] = df_viagem[coluna].apply(pd.to_numeric, errors='coerce')


Tratamento dos dados do dataframe passagens

In [ ]:
# Tratamento para as colunas com valores monetarios
colunas_to_numeric = ["Valor da passagem","Taxa de serviço"]
for coluna in colunas_to_numeric:
    df_passagem[coluna] = df_passagem[coluna].str.replace(',', '.')
    df_passagem[coluna] = df_passagem[coluna].apply(pd.to_numeric, errors='coerce')

'''# Copiando para novas linhas onde as colunas de volta não estão vazias
for index, row in df_passagem.iterrows():
    if row[9] != 'Sem informação':

        row[15] = row[15]/2
        row[16] = row[16]/2

        df_passagem.loc[index] = row

        nova_linha = row.copy()

        nova_linha = [nova_linha[0], None, nova_linha[2], nova_linha[9], nova_linha[10], nova_linha[11], nova_linha[12], nova_linha[13], nova_linha[14], None,None,None,None,None,None, nova_linha[15], nova_linha[16], nova_linha[17], nova_linha[18]]

        df_passagem.loc[len(df_passagem)] = nova_linha

        tempo de execucao = 90m 55.4s'''

# Removendo colunas desnecessárias
colunas_to_remove = ['Número da Proposta (PCDP)', 'País - Origem volta', 'UF - Origem volta', 'Cidade - Origem volta', 'Pais - Destino volta', 'UF - Destino volta', 'Cidade - Destino volta']
df_passagem.drop(colunas_to_remove, axis=1, inplace=True)

# Formatando data e hora para formato correto
df_passagem['Data da emissão/compra'] = pd.to_datetime(df_passagem['Data da emissão/compra'], format='%d/%m/%Y', errors='coerce')
df_passagem['Hora da emissão/compra'] = pd.to_datetime(df_passagem['Hora da emissão/compra'], format='%H:%M', errors='coerce').dt.time
 

In [61]:
df_viagem

,Identificador do processo de viagem,Número da Proposta (PCDP),Situação,Viagem Urgente,Justificativa Urgência Viagem,CPF viajante,Nome,Cargo,Motivo,Valor devolução,Valor outros gastos
0,18288418,000007/23-1C,Realizada,NÃO,Por necessidade do serviço.,***.621.358-**,ANDRE LUIS COSTA PITANGUEIRA,NaN,Participar da revisão do COE Manual (Contingen...,0.0,1095.35
1,18296348,000070/23,Realizada,NÃO,A efetivação do empenho no Siafi e a respectiv...,***.407.547-**,LIDIANY MIRANDA FERRAZ NUNES,AUXILIAR DE BIBLIIOTECA,Participação das aulas de Mestrado Interinstit...,0.0,0.00
2,18302983,000001/23,Realizada,NÃO,Sem informação,***.855.388-**,GREGOR GASPAR,NaN,PARTICIPAR DO CURSO DE PILOTO DE GRIPEN C/D.,0.0,0.00
3,18303291,000002/23-2C,Realizada,NÃO,VIAGEM COM DIÁRIA.,ID001000345,Informações protegidas por sigilo,Informações protegidas por sigilo,"Deslocamento de um(a) Agente Administrativo, c...",0.0,0.00
4,18306758,000002/23,Realizada,NÃO,Sem informação,***.295.391-**,VITOR LUIS MARTINS FARIA,NaN,PARTICIPAR DO CURSO DE PILOTO DE GRIPEN C/D.,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...
699819,2023001377,Sem informaçã,Realizada,NÃO,Sem informação,***.317.891-**,CARLOS EDUARDO DE ANDRADE BRANDT SILVA,ANALISTA DO BANCO CENTRAL,Participa como painelista no evento Score Summ...,0.0,0.00
699820,2023001379,Sem informaçã,Realizada,NÃO,Sem informação,***.061.486-**,ROGERIO ANTONIO LUCCA,ANALISTA DO BANCO CENTRAL,Acompanhar o Diretor em reunião com associados...,0.0,0.00
699821,2023001394,Sem informaçã,Realizada,NÃO,Sem informação,***.827.438-**,GABRIEL MURICCA GALIPOLO,DIRETOR DO BANCO CENTRAL,Cumpre agenda de reuniões em Brasília.,0.0,0.00
699822,2023001396,Sem informaçã,Realizada,NÃO,Sem informação,***.827.438-**,GABRIEL MURICCA GALIPOLO,DIRETOR DO BANCO CENTRAL,Cumpre agenda de reuniões presenciais em Brasília,0.0,0.00
